In [ ]:
import torch.nn as nn

class BrainTumorCNN(nn.Module):
    def __init__(self):
        super(BrainTumorCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 16 * 16, 64),
            nn.ReLU(),
            nn.Linear(64, 4)  
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x


In [ ]:
from google.colab import files
import os

uploaded = files.upload()

for filename in uploaded.keys():
    print(f'User uploaded file "{filename}"')
    if filename.endswith('.zip'):
        !unzip -q "{filename}" -d "/content/uploaded_dataset"
        dataset_path = "/content/uploaded_dataset" 
        print(f"Dataset unzipped to: {dataset_path}")
    else:
        dataset_path = f"/content/{filename}"
        print(f"Assuming uploaded file is the dataset directory: {dataset_path}")
if not os.path.exists(dataset_path):
    print(f"Error: Dataset path '{dataset_path}' still does not exist.")
else:
    print(f"Dataset found at: {dataset_path}")

Saving brain_tumor_dataset.zip to brain_tumor_dataset.zip
User uploaded file "brain_tumor_dataset.zip"
Dataset unzipped to: /content/uploaded_dataset
Dataset found at: /content/uploaded_dataset


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

# Hyperparameters
batch_size = 32
epochs = 10
learning_rate = 0.001

# Image transforms (grayscale + resize)
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load dataset
# Use the dataset_path variable set in the previous cell
if not os.path.exists(dataset_path):
    print(f"Error: Dataset path '{dataset_path}' not found. Please upload the dataset.")
else:
    train_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Device & model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = BrainTumorCNN().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        accuracy = 100 * correct / total
        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%")

    # Save model
    torch.save(model.state_dict(), 'brain_tumor_model.pth')
    print("✅ Model saved as brain_tumor_model.pth")

Epoch 1/10, Loss: 0.2461, Accuracy: 87.35%
Epoch 2/10, Loss: 0.0000, Accuracy: 100.00%
Epoch 3/10, Loss: 0.0000, Accuracy: 100.00%
Epoch 4/10, Loss: 0.0000, Accuracy: 100.00%
Epoch 5/10, Loss: 0.0000, Accuracy: 100.00%
Epoch 6/10, Loss: 0.0000, Accuracy: 100.00%
Epoch 7/10, Loss: 0.0000, Accuracy: 100.00%
Epoch 8/10, Loss: 0.0000, Accuracy: 100.00%
Epoch 9/10, Loss: 0.0000, Accuracy: 100.00%
Epoch 10/10, Loss: 0.0000, Accuracy: 100.00%
✅ Model saved as brain_tumor_model.pth
